# Bi-directional Elimination

In [4]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import absolute
from matplotlib import pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix,accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import warnings
warnings.filterwarnings('ignore')
import time
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

In [6]:
data=pd.read_csv("Group_2_data_cleaned.csv",index_col=0)
X=np.array(data.iloc[0:,3:12])
X1=data.iloc[0:,2:11]
Y=np.array(data.iloc[0:,11:])
data.head()

,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
Record_ID,,,,,,,,,,,,
1,732,_***i,0.200000,0.400000,0.0,0.000028,0.0,0.993593,0.000028,0.666667,5,0
2,732,g***r,0.024390,0.200000,0.0,0.013123,0.0,0.993593,0.013123,0.944444,5,0
3,732,t***p,0.142857,0.200000,0.0,0.003042,0.0,0.993593,0.003042,1.000000,5,0
4,732,7***n,0.100000,0.200000,0.0,0.097477,0.0,0.993593,0.097477,1.000000,5,0
5,900,z***z,0.051282,0.222222,0.0,0.001318,0.0,0.000000,0.001242,0.500000,7,0


In [7]:
print(data.describe())
print(data.info())

        Auction_ID  Bidder_Tendency  Bidding_Ratio  Successive_Outbidding  \
count  6321.000000      6321.000000    6321.000000            6321.000000   
mean   1241.388230         0.142541       0.127670               0.103781   
std     735.770789         0.197084       0.131530               0.279698   
min       5.000000         0.000000       0.011765               0.000000   
25%     589.000000         0.027027       0.043478               0.000000   
50%    1246.000000         0.062500       0.083333               0.000000   
75%    1867.000000         0.166667       0.166667               0.000000   
max    2538.000000         1.000000       1.000000               1.000000   

       Last_Bidding  Auction_Bids  Starting_Price_Average  Early_Bidding  \
count   6321.000000   6321.000000             6321.000000    6321.000000   
mean       0.463119      0.231606                0.472821       0.430683   
std        0.380097      0.255252                0.489912       0.380785   
mi

In [8]:
LE=LabelEncoder()
X[:,0]=LE.fit_transform(X[:,0]) #Select labels from variables
# #Split train and test data. Test data size is 50%, training set is 50%
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 0) 
# #The data set is normalized or 'standardized' for a better performaing KNN for both train and test data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#define K-fold cross-validation method to use
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [9]:
X1_train= pd.DataFrame(X_train, columns = X1.columns)
X1_train

,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration
0,-0.795875,-0.376591,0.403943,1.418303,1.063657,0.484108,-0.843153,-0.646002,-0.354057
1,-0.425545,-0.376591,0.409994,-0.922455,-0.965608,0.490314,-0.843153,-1.460430,-0.354057
2,0.456711,-0.376591,1.362000,-0.922455,-0.965608,1.439736,1.454297,0.982855,-0.354057
3,-0.240380,-0.376591,-0.777558,0.169899,1.063657,-0.694015,-0.843153,0.982855,-0.354057
4,0.456711,-0.376591,0.043186,-0.922455,-0.965608,0.081958,0.879934,-1.460430,-0.354057
...,...,...,...,...,...,...,...,...,...
3155,-0.839443,-0.376591,0.500425,1.545691,1.063657,-0.974836,-0.843153,0.982855,-0.354057
3156,0.663660,-0.376591,-1.213852,-0.922455,-0.965608,-1.129126,1.105577,0.168427,-0.354057
3157,-1.068176,-0.376591,0.709636,0.972444,1.063524,0.789143,-0.843153,-1.460430,-0.354057
3158,-0.327516,-0.376591,0.618211,0.377966,1.074495,0.697918,-0.843153,0.982855,-0.354057


In [10]:
features = X1_train.columns
features = features.to_list()
features

['Bidder_Tendency',
 'Bidding_Ratio',
 'Successive_Outbidding',
 'Last_Bidding',
 'Auction_Bids',
 'Starting_Price_Average',
 'Early_Bidding',
 'Winning_Ratio',
 'Auction_Duration']

In [11]:
X1.shape[1]

9

In [12]:
# Sequential Forward Floating Selection(sffs)
model=svm.SVC(kernel='linear')

sffs = SFS(model,
           k_features=(5,9),
           cv=5,
           floating=True)
         
sffs.fit(X1_train, y_train)
sffs.k_feature_names_

('Bidder_Tendency',
 'Bidding_Ratio',
 'Successive_Outbidding',
 'Last_Bidding',
 'Auction_Duration')

In [109]:
# Sequential Forward Floating Selection(sffs)
model=svm.SVC(kernel='poly')

sffs = SFS(model,
         k_features=(5,9),
         cv=5)
         
sffs.fit(X1_train, y_train)
sffs.k_feature_names_

('Bidder_Tendency',
 'Bidding_Ratio',
 'Successive_Outbidding',
 'Last_Bidding',
 'Auction_Duration')

### Bi-directional Elimination Variable Selection

##### 'Bidder_Tendency', 'Bidding_Ratio', 'Successive_Outbidding', 'Last_Bidding', 'Auction_Duration'

In [110]:
#Variable Selection
X_train_VS=X_train[0:,(0,1,2,3,8)]
y_train_VS=y_train
X_test_VS=X_test[0:,(0,1,2,3,8)]
y_test_VS=y_test

#### Best Model Part 2 (SVM LINEAR)

In [111]:
model = LinearSVC(penalty='l2', dual=False, max_iter=10000)
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}

start_time = time.time()
# Set up k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
# Set up GridSearchCV to perform model selection using k-fold cross-validation
clf = GridSearchCV(model, param_grid=param_grid, cv=kfold)
clf.fit(X_train, y_train)
end_time = time.time()
duration1=end_time-start_time
# Get the best model
best_model = clf.best_estimator_

# Evaluate the best model on the testing data
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {round(accuracy*100,2)}%")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2838
           1       1.00      1.00      1.00       323

    accuracy                           1.00      3161
   macro avg       1.00      1.00      1.00      3161
weighted avg       1.00      1.00      1.00      3161

Confusion Matrix:
 [[2838    0]
 [   0  323]] 

Accuracy: 100.0%


#### Best Model Part 3 (SVM POLY)

In [112]:
# SVM Nonlinear

# Set up the pipeline
svm_pipe = make_pipeline(StandardScaler(), SVC())

# Set up the grid search
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['poly', 'rbf', 'sigmoid'],
    'svc__gamma': ['scale', 'auto']
}
clf = GridSearchCV(svm_pipe, param_grid, cv=5)
# Fit the model and time it
clf.fit(X_train_VS, y_train)

# Use the best parameters to fit a new SVM model
best_params = clf.best_params_
svm_pipe = make_pipeline(StandardScaler(), SVC(C=best_params['svc__C'], kernel=best_params['svc__kernel'], gamma=best_params['svc__gamma']))
start_time = time.time()
svm_pipe.fit(X_train_VS, y_train)
end_time = time.time()

In [113]:
y_pred = svm_pipe.predict(X_test_VS)
print(classification_report(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_pred,y_test),'\n')
print('Accuracy:', round(accuracy_score(y_pred,y_test)*100,2),'%')

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2838
           1       1.00      0.99      1.00       323

    accuracy                           1.00      3161
   macro avg       1.00      1.00      1.00      3161
weighted avg       1.00      1.00      1.00      3161

Confusion Matrix:
 [[2838    2]
 [   0  321]] 

Accuracy: 99.94 %
